In [ ]:
import pickle
import time
import copy
import openpyxl
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.colors import ListedColormap
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from scipy.optimize import minimize
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score
import wntr

# Data loading

The data frame *df_leaks* holds pressure **residuals** for different times (along rows) and different sensors (along columns '0', '3', '30', ...). Moreover, the sensitive features and columns 'y_group1', 'y_group2' and 'y_group3' are binary labels telling us whether (1) or whether not (0) a leak is active for that time in group j for "j = 1,2,3". Additionally, the overall label and column 'y' is a binary label telling uns whether (1) or whether not (0) a leak is active in the WDN in general.
Additionally (and differently to the FairnessExploration_Hanoi and FairnessExploration_Hanoi_extended files), the column 'dia' gives information about the leak size.

The data frame *df_information* holds information about the leaks appearing in *df_leaks*. Each leak setting has the main characteristics 'node ID' and 'diameter'. 
Differently to the Hanoi sensor data, the data frame df_information is created based on the data frame df_leaks and not the data frame df_leaks is created based on the data frame df_information. As L-Town is a much larger water distribution network, not each node in the WDN is considered as a leaky node in the data. *df_information* holds information about each existing setting (along rows), such as 

- 'group' (areal group to which the leaky node belongs to),
- 'node ID' (location of the leak, but only placeholders - true node ID might differ),
- 'diameter' (size of the leak),
- 'setting start ID' (time index in the *df_leaks* at which the setting starts),
- 'leak start ID' (time index in the *df_leaks* at which the leak starts),
- 'leak end ID' (time index in the *df_leaks* at which the leak ends),
- 'setting end ID' (time index in the *df_leaks* at which the setting ends)

(along columns).

The data frame *df_noleaks* is not available in this setting. As this data frame is only used for visualization, this is not an issue. 

**Important**: Note that due to the fact the data frame df_leaks holds pressure residuals instead of pressure measurements, the regression part can be skipped in this notebook. Also due to the fact that there is no df_noleaks data frame, most of the visualizations regarding the pressure (residual) measurements are not displayed in this notebook.

In [ ]:
def LeakInformation(df):
    
    # ----- leak information in data frame style
    # --- initialize data frame
    data = {'group': list(),
            'node ID': list(), 
            'diameter': list(), 
            'setting start ID': list(),
            'leak start ID': list(), 
            'setting end ID': list(),
            'leak end ID': list()}
    df_information = pd.DataFrame(data=data)

     # --- fill data frame
    idx_info = 1
    for idx_1 in df.index[:-2]:

        # find index where leaky scenario starts
        dia_end =  df.loc[idx_1,'dia']
        dia_start =  df.loc[idx_1+1,'dia']
        if dia_end != dia_start:

            idx_start_scenario = idx_1+1

            # find index where leak in leaky scenario starts and ends
            for idx_2 in df.index[idx_start_scenario:-2]:

                y_end = df.loc[idx_2,'y']
                y_start = df.loc[idx_2 + 1,'y']
                if y_end == 0 and y_start == 1:
                    idx_start_leak = idx_2 + 1
                if y_end == 1 and y_start == 0:
                    idx_end_scenario = idx_2
                    idx_end_leak = idx_2 + 1
                    break
                else:
                    idx_end_scenario = idx_2
                    idx_end_leak = idx_2

            # access all information of the scenario at the end of the scenario
            # as at the end, the leak is present
            for group in ['y_group1','y_group2','y_group3']:
                if df.loc[idx_end_scenario, group] == 1:
                    df_information.loc[idx_info,'group'] = group[2:]
        
            df_information.loc[idx_info,'node ID'] = str(int(1 + (idx_info-1) / 3)) # fake nodeIDs
            df_information.loc[idx_info,'diameter'] = df.loc[idx_end_scenario,'dia']
            df_information.loc[idx_info,'setting start ID'] = idx_start_scenario
            df_information.loc[idx_info,'leak start ID'] = idx_start_leak
            df_information.loc[idx_info,'setting end ID'] = idx_end_scenario
            df_information.loc[idx_info,'leak end ID'] = idx_end_leak

            idx_info += 1

    df_information['Y = 0 on left side'] = (df_information['leak start ID'] - 1) - df_information['setting start ID'] + 1
    df_information['Y = 1'] = (df_information['leak end ID'] - 1) - df_information['leak start ID'] + 1
    df_information['Y = 0 on right side'] = df_information['setting end ID'] - df_information['leak end ID'] + 1
    df_information['Y = 0'] = df_information['Y = 0 on left side'] + df_information['Y = 0 on right side']

    return df_information

In [ ]:
df_leaks = pd.read_csv('../2_DataGeneration/L-Town/data_leaks_LTown.csv')
df_leaks = df_leaks.drop(['Unnamed: 0'], axis=1)
df_leaks = df_leaks.rename(columns = {"A": "y_group1", 
                                      "B": "y_group2",
                                      "C": "y_group3"})
# transform m in cm
df_leaks['dia'] = df_leaks['dia']*100
df_information = LeakInformation(df_leaks)

In [ ]:
df_leaks

In [ ]:
df_information

# Pipeline definition

In [ ]:
%run ./FairnessExploration_PipelineDefinition.ipynb

# Pipeline application

## Variables

In [ ]:
# define the classifiers used per sensor node
classifier = ThresholdClassification
classifier_approx = ThresholdClassificationApproximation

In [ ]:
# access node ids
df_nodes = pd.read_csv('../2_DataGeneration/L-Town/data_nodes_LTown.csv')
df_nodes = df_nodes.drop(['Unnamed: 0'], axis=1)
node_ids = list(df_nodes.columns)
#print('Given nodes: {}'.format(node_ids))

# access sensor ids
df_sensors = pd.read_csv('../2_DataGeneration/L-Town/data_leaks_LTown.csv')
df_sensors = df_sensors.drop(['Unnamed: 0','y','A','B','C','dia'], axis=1)
sensor_ids = list(df_sensors.columns)
sensor_ids_n = ['n'+str(int(sensor)+1) for sensor in sensor_ids]
print('Given sensors: {}'.format(sensor_ids_n))

# access sensitive features
sensitive_features = list(df_leaks.columns[33:36])
print('Given sensitive features: {}'.format(sensitive_features))

## Visualization - Network

In [ ]:
groups_per_node = dict()
for node in node_ids:
    if df_nodes.loc[0,node] == 1.0:
        groups_per_node[node] = 'group1'
    if df_nodes.loc[1,node] == 2.0:
        groups_per_node[node] = 'group2'
    if df_nodes.loc[2,node] == 3.0:
        groups_per_node[node] = 'group3'

In [ ]:
wn_ltown = wntr.network.WaterNetworkModel('../1_FeatureGeneration/models/LTown.inp') 

plot_network_LTown(node_ids=node_ids,
                   sensor_ids=sensor_ids_n,
                   groups_per_node=groups_per_node,
                   wn=wn_ltown,
                   name='L-Town',
                   save_figs=False)

## Preprocessing for Classification - Compute Residuals

In [ ]:
# access residuals based on true data and predicted data
X_clas = df_leaks.loc[:,sensor_ids]

X_sen = df_leaks.loc[:,sensitive_features]
y_clas = df_leaks.loc[:,['y']]

In [ ]:
#X_clas

In [ ]:
#X_sen

In [ ]:
#y_clas

## Visualization - True and Predicted Pressure

In [ ]:
# plot pressure *residuals*
plot_data_per_setting(X_clas,
                      df_information=df_information,
                      sensor_ids=sensor_ids,
                      node_ids=[str(x) for x in range(1,31)],
                      diameters=[2.3],
                      setting_ids=None,
                      #thresholds={'3':0.5, '10':2, '23':0.2, '25':0.5},
                      time_puffer=1000,
                      show_legend=True,
                      zoom_leak=True,
                      print_report=False)

## Classification - Leak Detector(s)

We now use the virtual sensors to predict the pressure even for times where a leak is active in the WDN. We make use of the residuals $|p_j(t_i) - f_j^r(p_{\neq j}(t_i))| \in \mathbb{R}$ to define a threshold-based classifier that predicts whether a leak is active (1) or not (0) at time $t_i$. 

In [ ]:
# create dictionary to store all results
# which are visualized at the end
results_fairness= dict()
results_nofairness = dict()
# define which fairness method
# should improve which non-fairness method
comparisons = {'DI+ACC-ndb-max':'ACC-ndb'}

**!!! Remark: !!!** Running the next blocks ("Diameter = 1.9", "Diameter = 2.3" and "Diameter = 2.7") takes 48-72 hrs. You can skip this part by loading the results in the "Load Results" block below.

### Diameter = 1.9

In [ ]:
# filter the training and test data according to the diameter
diameter = 1.9
results = filter_diameter_LTown(X_clas, y_clas, X_sen, diameter=diameter, df_leaks=df_leaks)
X_clas_train, X_clas_test, y_clas_train, y_clas_test, X_sen_train, X_sen_test = results
print(X_sen_train.sum())
print(X_sen_test.sum())

# create dictionary to store all results 
# which are visualized at the end
results_d5 = dict()
results_fairness[diameter] = dict()
results_nofairness[diameter] = dict()

#### Method: Choose hyperparameter

In [ ]:
model_clas = ETC_hyperparameter()
model_clas.fit(X_clas_train, factor=0.25, print_coeff=True)
acc,eo,di,TPRs = evaluate(model_clas)

In [ ]:
max_TPR = max(list(TPRs.values()))
min_TPR = min(list(TPRs.values()))
print('{} & {} & {} & {} & {} & {} & {}'.format(round(acc, 4),
                                                round(max_TPR, 4),
                                                round(min_TPR, 4),
                                                round(di, 4),
                                                round(eo, 4),
                                                round((1-eo/max_TPR), 4),
                                                round((1-di)*max_TPR, 4)))

In [ ]:
results_d5[model_clas] = {'acc':acc,'eo':eo,'di':di,'TPRs':TPRs}
results_nofairness[diameter][model_clas.alias] = dict()
results_nofairness[diameter][model_clas.alias]['acc'] = acc
results_nofairness[diameter][model_clas.alias]['eo'] = eo
results_nofairness[diameter][model_clas.alias]['di']= di

In [ ]:
# define the starting point for ACC algorithm
start_thresholds = model_clas.thresholds

#### Method: Optimize ACC (ndb.)

In [ ]:
model_clas = ETC_optimizeACC_ndb(alias='ACC-ndb')
model_clas.fit(X_clas_train, 
               X_sen_train,
               y_clas_train,
               start_thresholds=start_thresholds,
               print_coeff=True)
acc,eo,di,TPRs = evaluate(model_clas)

In [ ]:
results_d5[model_clas] = {'acc':acc,'eo':eo,'di':di,'TPRs':TPRs}
results_nofairness[diameter][model_clas.alias] = dict()
results_nofairness[diameter][model_clas.alias]['acc'] = acc
results_nofairness[diameter][model_clas.alias]['eo'] = eo
results_nofairness[diameter][model_clas.alias]['di']= di

In [ ]:
# define the starting point for DI+ACC-ndb-max algorithm
start_thresholds = model_clas.thresholds

In [ ]:
_,_,_,_,_,acc_best,_ = model_clas.score(X_clas_train, 
                                        y_clas_train,
                                        print_all_scores=False)
acc_best

#### Method: Optimize DI (ndb., max-barrier)

In [ ]:
# model_clas = ETC_optimizeDI_ndb(alias='DI+ACC-ndb-max')
# model_clas.fit(X_clas_train,
#                X_sen_train,
#                y_clas_train,
#                start_thresholds=start_thresholds,
#                mu=100, #100
#                lamb=0.02, #0.00, 0.01, 0.02!, 0.03, ..., 0.33
#                barrier='max',
#                acc_best=acc_best,
#                print_coeff=True)
# acc,eo,di,TPRs = evaluate(model_clas)

In [ ]:
# --- specify hyperparameters and model class
start_hyper = 0.33
model_clas = ETC_optimizeDI_ndb(alias='DI+ACC-ndb-max')

# --- define constant for stopping criterium
comparison_algo = comparisons[model_clas.alias]
di_nofairness = results_nofairness[diameter][comparison_algo]['di']
print('disparate impact of approx. {} '\
      'from {} algorithm is used as a stopping criterium.'.format(round(di_nofairness,5),
                                                                        comparison_algo))

# --- test model for different hyperparameters
Cs = [round(start_hyper-i*0.01,2) for i in range(0,40) if round(start_hyper-i*0.01,2)>=0]
print('\nHyperparameters to test:\n', Cs)
results_fairness[diameter][model_clas.alias] = dict()
results_fairness[diameter][model_clas.alias]['ACCs'] = list()
results_fairness[diameter][model_clas.alias]['EOs'] = list()
results_fairness[diameter][model_clas.alias]['DIs'] = list()
results_fairness[diameter][model_clas.alias]['Lambdas'] = list()
for lamb in Cs:
    print('\nlambda:', lamb)
    # --- train model for fixed hyperparameters
    model_clas = ETC_optimizeDI_ndb(alias='DI+ACC-ndb-max')
    model_clas.fit(X_clas_train,
                   X_sen_train,
                   y_clas_train,
                   start_thresholds=start_thresholds,
                   mu=100, #100
                   lamb=lamb, #0.00, 0.01, 0.02!, 0.03, ..., 0.33
                   barrier='max',
                   acc_best=acc_best,
                   print_coeff=False)
    # --- evaluate model for fixed hyperparameters
    acc,eo,di,TPRs = evaluate(model_clas)
    # --- store evaluation until model is as unfair as comparison model
    if di <= di_nofairness:
        print('\nHyperparameter {} and larger were not used'.format(lamb))
        break
    results_d5[model_clas] = {'acc':acc,'eo':eo,'di':di,'TPRs':TPRs}
    results_fairness[diameter][model_clas.alias]['ACCs'].append(acc)
    results_fairness[diameter][model_clas.alias]['EOs'].append(eo)
    results_fairness[diameter][model_clas.alias]['DIs'].append(di)
    results_fairness[diameter][model_clas.alias]['Lambdas'].append(lamb)

### Diameter = 2.3

In [ ]:
# filter the training and test data according to the diameter
diameter = 2.3
results = filter_diameter_LTown(X_clas, y_clas, X_sen, diameter=diameter, df_leaks=df_leaks)
X_clas_train, X_clas_test, y_clas_train, y_clas_test, X_sen_train, X_sen_test = results
print(X_sen_train.sum())
print(X_sen_test.sum())

# create dictionary to store all results 
# which are visualized at the end
results_d10 = dict()
results_fairness[diameter] = dict()
results_nofairness[diameter] = dict()

#### Method: Choose hyperparameter

In [ ]:
model_clas = ETC_hyperparameter()
model_clas.fit(X_clas_train, factor=0.19, print_coeff=True)
acc,eo,di,TPRs = evaluate(model_clas)

In [ ]:
max_TPR = max(list(TPRs.values()))
min_TPR = min(list(TPRs.values()))
print('{} & {} & {} & {} & {} & {} & {}'.format(round(acc, 4),
                                                round(max_TPR, 4),
                                                round(min_TPR, 4),
                                                round(di, 4),
                                                round(eo, 4),
                                                round((1-eo/max_TPR), 4),
                                                round((1-di)*max_TPR, 4)))

In [ ]:
results_d10[model_clas] = {'acc':acc,'eo':eo,'di':di,'TPRs':TPRs}
results_nofairness[diameter][model_clas.alias] = dict()
results_nofairness[diameter][model_clas.alias]['acc'] = acc
results_nofairness[diameter][model_clas.alias]['eo'] = eo
results_nofairness[diameter][model_clas.alias]['di']= di

In [ ]:
# define the starting point for ACC algorithm
start_thresholds = model_clas.thresholds

#### Method: Optimize ACC (ndb.)

In [ ]:
model_clas = ETC_optimizeACC_ndb(alias='ACC-ndb')
model_clas.fit(X_clas_train, 
               X_sen_train,
               y_clas_train,
               start_thresholds=start_thresholds,
               print_coeff=True)
acc,eo,di,TPRs = evaluate(model_clas)


In [ ]:
results_d10[model_clas] = {'acc':acc,'eo':eo,'di':di,'TPRs':TPRs}
results_nofairness[diameter][model_clas.alias] = dict()
results_nofairness[diameter][model_clas.alias]['acc'] = acc
results_nofairness[diameter][model_clas.alias]['eo'] = eo
results_nofairness[diameter][model_clas.alias]['di']= di

In [ ]:
# define the starting point for DI+ACC-ndb-max algorithm
start_thresholds = model_clas.thresholds

In [ ]:
_,_,_,_,_,acc_best,_ = model_clas.score(X_clas_train, 
                                        y_clas_train,
                                        print_all_scores=False)
acc_best

#### Method: Optimize DI (ndb., max-barrier)

In [ ]:
#model_clas = ETC_optimizeDI_ndb(alias='DI+ACC-ndb-max')
#model_clas.fit(X_clas_train,
#               X_sen_train,
#               y_clas_train,
#               start_thresholds=start_thresholds,
#               mu=100, #100
#               lamb=0.0, #0.00!, 0.01, ..., 0.46
#               barrier='max',
#               acc_best=acc_best,
#               print_coeff=True)
#acc,eo,di,TPRs = evaluate(model_clas)

In [ ]:
# --- specify hyperparameters and model class
start_hyper = 0.46
model_clas = ETC_optimizeDI_ndb(alias='DI+ACC-ndb-max')

# --- define constant for stopping criterium
comparison_algo = comparisons[model_clas.alias]
di_nofairness = results_nofairness[diameter][comparison_algo]['di']
print('disparate impact of approx. {} '\
      'from {} algorithm is used as a stopping criterium.'.format(round(di_nofairness,5),
                                                                        comparison_algo))

# --- test model for different hyperparameters
Cs = [round(start_hyper-i*0.01,2) for i in range(0,50) if round(start_hyper-i*0.01,2)>=0]
print('\nHyperparameters to test:\n', Cs)
results_fairness[diameter][model_clas.alias] = dict()
results_fairness[diameter][model_clas.alias]['ACCs'] = list()
results_fairness[diameter][model_clas.alias]['EOs'] = list()
results_fairness[diameter][model_clas.alias]['DIs'] = list()
results_fairness[diameter][model_clas.alias]['Lambdas'] = list()
for lamb in Cs:
    print('\nlambda:', lamb)
    # --- train model for fixed hyperparameters
    model_clas = ETC_optimizeDI_ndb(alias='DI+ACC-ndb-max')
    model_clas.fit(X_clas_train,
                   X_sen_train,
                   y_clas_train,
                   start_thresholds=start_thresholds,
                   mu=100, #100
                   lamb=lamb, #0.00!, 0.01, ..., 0.46
                   barrier='max',
                   acc_best=acc_best,
                   print_coeff=False)
    # --- evaluate model for fixed hyperparameters
    acc,eo,di,TPRs = evaluate(model_clas)
    # --- store evaluation until model is as unfair as comparison model
    if di <= di_nofairness:
        print( 
            '\nHyperparameter {} and larger were not used'.format(lamb))
        break
    results_d10[model_clas] = {'acc':acc,'eo':eo,'di':di,'TPRs':TPRs}
    results_fairness[diameter][model_clas.alias]['ACCs'].append(acc)
    results_fairness[diameter][model_clas.alias]['EOs'].append(eo)
    results_fairness[diameter][model_clas.alias]['DIs'].append(di)
    results_fairness[diameter][model_clas.alias]['Lambdas'].append(lamb)

### Diameter = 2.7

In [ ]:
# filter the training and test data according to the diameter
diameter = 2.7
results = filter_diameter_LTown(X_clas, y_clas, X_sen, diameter=diameter, df_leaks=df_leaks)
X_clas_train, X_clas_test, y_clas_train, y_clas_test, X_sen_train, X_sen_test = results
print(X_sen_train.sum())
print(X_sen_test.sum())

# create dictionary to store all results 
# which are visualized at the end
results_d15 = dict()
results_fairness[diameter] = dict()
results_nofairness[diameter] = dict()

#### Method: Choose hyperparameter

In [ ]:
model_clas = ETC_hyperparameter()
model_clas.fit(X_clas_train, factor=0.15, print_coeff=True)
acc,eo,di,TPRs = evaluate(model_clas)

In [ ]:
max_TPR = max(list(TPRs.values()))
min_TPR = min(list(TPRs.values()))
print('{} & {} & {} & {} & {} & {} & {}'.format(round(acc, 4),
                                                round(max_TPR, 4),
                                                round(min_TPR, 4),
                                                round(di, 4),
                                                round(eo, 4),
                                                round((1-eo/max_TPR), 4),
                                                round((1-di)*max_TPR, 4)))

In [ ]:
results_d15[model_clas] = {'acc':acc,'eo':eo,'di':di,'TPRs':TPRs}
results_nofairness[diameter][model_clas.alias] = dict()
results_nofairness[diameter][model_clas.alias]['acc'] = acc
results_nofairness[diameter][model_clas.alias]['eo'] = eo
results_nofairness[diameter][model_clas.alias]['di']= di

In [ ]:
# define the starting point for ACC algorithm
start_thresholds = model_clas.thresholds

#### Method: Optimize ACC (ndb.)

In [ ]:
model_clas = ETC_optimizeACC_ndb(alias='ACC-ndb')
model_clas.fit(X_clas_train, 
               X_sen_train,
               y_clas_train,
               start_thresholds=start_thresholds,
               print_coeff=True)
acc,eo,di,TPRs = evaluate(model_clas)

In [ ]:
results_d15[model_clas] = {'acc':acc,'eo':eo,'di':di,'TPRs':TPRs}
results_nofairness[diameter][model_clas.alias] = dict()
results_nofairness[diameter][model_clas.alias]['acc'] = acc
results_nofairness[diameter][model_clas.alias]['eo'] = eo
results_nofairness[diameter][model_clas.alias]['di']= di

In [ ]:
# define the starting point for DI+ACC-ndb-max algorithm
start_thresholds = model_clas.thresholds

In [ ]:
_,_,_,_,_,acc_best,_ = model_clas.score(X_clas_train, 
                                        y_clas_train,
                                        print_all_scores=False)
acc_best

#### Method: Optimize DI (ndb., max-barrier)

In [ ]:
#model_clas = ETC_optimizeDI_ndb(alias='DI+ACC-ndb-max')
#model_clas.fit(X_clas_train,
#               X_sen_train,
#               y_clas_train,
#               start_thresholds=start_thresholds,
#               mu=100, #100
#               lamb=0.01, #0.0, 0.1!, 0.2 ..., 0.5
#               barrier='max',
#               acc_best=acc_best,
#               print_coeff=True)
#acc,eo,di,TPRs = evaluate(model_clas)

In [ ]:
# --- specify hyperparameters and model class
start_hyper = 0.5
model_clas = ETC_optimizeDI_ndb(alias='DI+ACC-ndb-max')

# --- define constant for stopping criterium
comparison_algo = comparisons[model_clas.alias]
di_nofairness = results_nofairness[diameter][comparison_algo]['di']
print('disparate impact of approx. {} '\
      'from {} algorithm is used as a stopping criterium.'.format(round(di_nofairness,5),
                                                                        comparison_algo))

# --- test model for different hyperparameters
Cs = [round(start_hyper-i*0.01,2) for i in range(0,60) if round(start_hyper-i*0.01,2)>=0]
print('\nHyperparameters to test:\n', Cs)
results_fairness[diameter][model_clas.alias] = dict()
results_fairness[diameter][model_clas.alias]['ACCs'] = list()
results_fairness[diameter][model_clas.alias]['EOs'] = list()
results_fairness[diameter][model_clas.alias]['DIs'] = list()
results_fairness[diameter][model_clas.alias]['Lambdas'] = list()
for lamb in Cs:
    print('\nlambda:', lamb)
    # --- train model for fixed hyperparameters
    model_clas = ETC_optimizeDI_ndb(alias='DI+ACC-ndb-max')
    model_clas.fit(X_clas_train,
                   X_sen_train,
                   y_clas_train,
                   start_thresholds=start_thresholds,
                   mu=100, #100
                   lamb=lamb, #0.0, 0.1!, 0.2 ..., 0.5
                   barrier='max',
                   acc_best=acc_best,
                   print_coeff=False)
    # --- evaluate model for fixed hyperparameters
    acc,eo,di,TPRs = evaluate(model_clas)
    # --- store evaluation until model is as unfair as comparison model
    if di <= di_nofairness:
        print('\nHyperparameter {} and larger were not used'.format(lamb))
        break
    results_d15[model_clas] = {'acc':acc,'eo':eo,'di':di,'TPRs':TPRs}
    results_fairness[diameter][model_clas.alias]['ACCs'].append(acc)
    results_fairness[diameter][model_clas.alias]['EOs'].append(eo)
    results_fairness[diameter][model_clas.alias]['DIs'].append(di)
    results_fairness[diameter][model_clas.alias]['Lambdas'].append(lamb)

### Store Results 
(as computation time of the previous blocks is about 48-72hrs)

In [ ]:
# save dictionaries to pkl file
with open('results_LTown_d5.pkl', 'wb') as file:
    pickle.dump(results_d5, file)
with open('results_LTown_d10.pkl', 'wb') as file:
    pickle.dump(results_d10, file)
with open('results_LTown_d15.pkl', 'wb') as file:
    pickle.dump(results_d15, file)
with open('results_LTown_results_fairness.pkl', 'wb') as file:
    pickle.dump(results_fairness, file)
with open('results_LTown_results_nofairness.pkl', 'wb') as file:
    pickle.dump(results_nofairness, file)

### Load Results
(in case you do not want to run the code above)

In [ ]:
# read dictionary from pkl file
with open('./results_L-Town/results_L-Town_d5.pkl', 'rb') as file:
    results_d5 = pickle.load(file)
with open('./results_L-Town/results_L-Town_d10.pkl', 'rb') as file:
    results_d10 = pickle.load(file)
with open('./results_L-Town/results_L-Town_d15.pkl', 'rb') as file:
    results_d15 = pickle.load(file)
with open('./results_L-Town/results_L-Town_results_fairness.pkl', 'rb') as file:
    results_fairness = pickle.load(file)
with open('./results_L-Town/results_L-Town_results_nofairness.pkl', 'rb') as file:
    results_nofairness = pickle.load(file)

### Visualize Results

In [ ]:
aliase = ['H','ACC-ndb','DI+ACC-ndb-max']
results_d5_final = dict()
for key in results_d5.keys():
    if key.alias in aliase:
        results_d5_final[key] = results_d5[key]
        
df1, df2, df3, fig_d5 = graphics_bars(results_d5_final,
                                      save_figs_d=1.9)

In [ ]:
aliase = ['H','ACC-ndb','DI+ACC-ndb-max']
results_d10_final = dict()
for key in results_d10.keys():
    if key.alias in aliase:
        results_d10_final[key] = results_d10[key]
        
df1, df2, df3, fig_d10 = graphics_bars(results_d10_final,
                                       save_figs_d=2.3)

In [ ]:
aliase = ['H','ACC-ndb','DI+ACC-ndb-max']
results_d15_final = dict()
for key in results_d15.keys():
    if key.alias in aliase:
        results_d15_final[key] = results_d15[key]

df1, df2, df3, fig_d15 = graphics_bars(results_d15_final,
                                       save_figs_d=2.7)

In [ ]:
graphics_scatter(results_fairness, 
                 results_nofairness,
                 comparisons,
                 horizontal=True,
                 save_figs=True)

In [ ]:
# this is just some work-around to keep the figure size equal to the Hanoi figures
results_fairness_copy = copy.deepcopy(results_fairness)
for diameter in results_fairness_copy.keys():
    for alias in ['Alias 1', 'Alias 2', 'Alias 3']:
        results_fairness_copy[diameter][alias] = results_fairness[diameter]['DI+ACC-ndb-max'].copy()

In [ ]:
graphics_lines(results_fairness_copy,
               results_nofairness,
               comparisons,
               columns='diameters',
               with_eo=True,
               save_figs=True)